# 5. Composição de Multiplas Visualizações

Na hora de visualizar distintos campos de dados, podemos nos sentir tentados a usar o máximo possível de canais de codificação visual: `x`,`y`,`cor`,`tamanho`,`formato` e etc. No entanto, conforme o número de canais de codificação aumenta, o gráfico pode rapidamente ficar truncado e difícil de ler. Em alternativa a sobrecarregar um único gráfico, podemos usar a _composição de múltiplos gráficos_ para facilitar comparações rápidas.

Nesse capítulo, nós vamos examinar uma variedade de operações para _Composição de Multiplas Visualizações_:

- _layer_: coloca gráficos compatíveis diretamente um em cima do outro,
- _facet_: particiona dados em múltiplos gráficos, organizados em linhas e colunas,
- _concatenate_: posiciona arbitrariamente gráficos na mesma formatação, e
- _repeat_: seleciona uma especificação de um gráfico de base e aplica em múltiplos campos de dados.

Nós vamos então ver como essas operações formam a _álgebra de composição visual_, na qual essas operações podem ser combinadas para criar uma variedade de complexas exibições de múltiplas visualizações.

_Esse notebook é parte do [Currículo de Visualização de Dados](https://github.com/uwdata/visualization-curriculum)._

In [1]:
import pandas as pd
import altair as alt

## 5.1 Dados de Clima

Nós vamos estar visualizando estatísticas de clima nas cidades de Seattle e Nova Iorque nos Estados Unidos. Vamos carregar o conjunto e espiar as primeiras e últimas 10 colunas:


In [2]:
clima = 'https://cdn.jsdelivr.net/npm/vega-datasets@1/data/weather.csv'

In [3]:
df = pd.read_csv(clima)
df.head(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
0,Seattle,2012-01-01,0.0,12.8,5.0,4.7,drizzle
1,Seattle,2012-01-02,10.9,10.6,2.8,4.5,rain
2,Seattle,2012-01-03,0.8,11.7,7.2,2.3,rain
3,Seattle,2012-01-04,20.3,12.2,5.6,4.7,rain
4,Seattle,2012-01-05,1.3,8.9,2.8,6.1,rain
5,Seattle,2012-01-06,2.5,4.4,2.2,2.2,rain
6,Seattle,2012-01-07,0.0,7.2,2.8,2.3,rain
7,Seattle,2012-01-08,0.0,10.0,2.8,2.0,sun
8,Seattle,2012-01-09,4.3,9.4,5.0,3.4,rain
9,Seattle,2012-01-10,1.0,6.1,0.6,3.4,rain


In [4]:
df.tail(10)

,location,date,precipitation,temp_max,temp_min,wind,weather
2912,New York,2015-12-22,4.8,15.6,11.1,3.8,fog
2913,New York,2015-12-23,29.5,17.2,8.9,4.5,fog
2914,New York,2015-12-24,0.5,20.6,13.9,4.9,fog
2915,New York,2015-12-25,2.5,17.8,11.1,0.9,fog
2916,New York,2015-12-26,0.3,15.6,9.4,4.8,drizzle
2917,New York,2015-12-27,2.0,17.2,8.9,5.5,fog
2918,New York,2015-12-28,1.3,8.9,1.7,6.3,snow
2919,New York,2015-12-29,16.8,9.4,1.1,5.3,fog
2920,New York,2015-12-30,9.4,10.6,5.0,3.0,fog
2921,New York,2015-12-31,1.5,11.1,6.1,5.5,fog


Nós vamos criar exibições de múltiplas visualizações para examinar dentro e entre as cidades.

## 5.2 Camadas

Uma das maneiras mais comuns de combinar gráficos é utilizando a função _layer_. Caso os gráficos tenham domínios compatíveis podemos juntá-los para formas _eixos compartilhados_. Caso os codificadores de `x` ou `y` não forem compatíveis teremos que criar um _gráfico de eixo duplo_ que se sobrepõe utilizando escalas e eixos diferentes.

### 5.2.1 Eixos compartilhados


Comecemos exibindo a temperátura média máxima e mínima por mês:

In [5]:
alt.Chart(clima).mark_area().encode(
  alt.X('month(date):T', title="Mês"),
  alt.Y('average(temp_max):Q', title='Temperatura média máxima, Temperatura média mínima'),
  alt.Y2('average(temp_min):Q')
)

alt.Chart(...)

_O gráfico nos mostra o intervalo de temperaturas médias a cada mês considerando todos os dados. No entanto, isso não nos diz muito já que está misturando os dados de Nova Yotk e Seattle._

Vmoa então dividir os dados baseados nas cidades por cores, mudando também a opcaidade para considerar áreas de sobreposição.

In [6]:
alt.Chart(clima).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title="Mês"),
  alt.Y('average(temp_max):Q', title='Temperatura média máxima, Temperatura média mínima'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N', title='Cidade')
)

alt.Chart(...)

_Podemos perceber que Seattle é mais temperado, tendo temperaturas mais amenas durante o verão e mais quente durante o inverno_

nesse caso, criamos um gráfico em camadas sem características mais complexas simplesmente dividindo as áreas por cor. Já que o gráfico acima nos mostra os interalos das temperaturas também é válido mostrar o meio do intervalo.

Vamos criar um gráfico mostrando o ponto médio da temperatura média. Usaremos a transformação `Calculate` para calcular os pontos médios entre a temperatura média mínima e máxima.

In [7]:
alt.Chart(clima).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_mid):Q', title= 'Temperatura do ponto médio'),
  alt.Color('location:N', title='Cidade')
)

alt.Chart(...)

Observação: Note o uso do `+datum.temp_min` na tranformação acima. Já que estamos carregando os dados direto de um arquivo CSV sem nenhuma especificação os valores de temperatura podem estar sendo representados internamente como strings. Adicionando `+` na frente dos valores os força a serem lidos como números.

Agora nós gostaríaoms de combinar os gráficos colocando os pontos médios por cima das áreas das temperaturas. Utilizando a sintax `chart1 + chart2`, nós podemos especificar um novo gráfico combinado em que `chart1` é a primeira camada e `chart2` é a segunda camada no topo:

In [8]:
tempMinMax = alt.Chart(clima).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_max):Q', title='Temperatura média máxima, Temperatura média mínima'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(clima).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_mid):Q', title='Temperatura do ponto médio'),
  alt.Color('location:N', title= 'Cidade')
)

tempMinMax + tempMid

alt.LayerChart(...)

_Agora temos um gráfico de várias camadas! Contido o título do eixo-y ficou meio confuso..._

Vamos personalizar os eixos para deixar nosso gráfico mais limpo. Se definirmos um título para uma camada isso valerá para todas



In [9]:
tempMinMax = alt.Chart(clima).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart(clima).mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T', title='Mês'),
  alt.Y('average(temp_mid):Q', title= 'Temperatura média °C'),
  alt.Color('location:N', title= 'Cidade')
)

tempMinMax + tempMid

alt.LayerChart(...)

_O que aconteceria se ambas camadas tivessem eixos personalizados? Modifique o código acima para descobrir_

Utilizamos acima `+` que é um atalho para o método `layer`. Podemos criar um gráfico idêntico usando o método `layer` diretamente:

In [10]:
alt.layer(tempMinMax, tempMid)

alt.LayerChart(...)

Perceba que a ordem das camadas é relevante para qual vai sobrepor qual. _Exeprimente mudar a ordem e ver os resultados. O que acontece?(Dica: preste atenção na cor da linha)_

### 5.2.2 Gráfico com eixos duplos

_Seattle tem fama de chuvosa, isso é merecido?_

Vamos olhar para a precipitação junto com a temperatura para analisar melhor. Vamos primeiramente montar um gráfico base para mostrar a precipitação em Seattle:

In [11]:
alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.Chart(...)

Para facilitar a comparação com os dados da temperatura vamos criar um gráfico de camadas. Isso é o que acontece se tentarmos sobrepor os gráficos como fizemos antes:

In [12]:
tempMinMax = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip)

alt.LayerChart(...)

_Os valores de precipitação são consideravelmente menores que os de temperatura, ou seja, usam um intervalo do eixo-y bem menor!_

Por padrâo, os gráficos em camadas usam um _domínio cmoporatilhado_: os valores dos eixos x e y são combinados por todas as camadas para obter uma extensão compratilhada adequada. Esse comportamento padrão assume que as camadas compratilham as mesmas unidades nos eixos. Entretanto, isso não se aplica a esse exemplo, já que estamos unindo valores de tempreatura (graus Celsius) e precipitação (polegadas)!

Se quisermos usar escalas diferentes no eixo y, precisamos especificar para o Altair que queremos diferenciar os dados por camada. Nesse caso, queremos que as escalas e domínios do eixo y sejam independentes e não compartilhados. O objeto `Chart` inclui o método `resolved_scale` que lida com isso deixando-nos escolher o que desejamos:

In [13]:
tempMinMax = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip).resolve_scale(y='independent')

alt.LayerChart(...)

Agora podemos perceber que o Outono é a estação mais chuvosa em Seattle (tendo seu pico em novembro), seguido de verôes mais secos.

Você pode ter notado um redundância nas especificações dos nossos gráficos: ambos usam a mesma base de dados e tem um filtro para usar apenas dados de Seattle. Se quiser pode simplificar o código especificando a base de dados e o filtro para o gráfico de maior nível das camadas. As outras camadas vão herdar os dados se não for especificado nada diferente:

In [14]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média °C'),
  alt.Y2('average(temp_min):Q')
)

precip = alt.Chart().mark_line(
  interpolate='monotone',
  stroke='grey'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(precipitation):Q', title='Precipitação')
)

alt.layer(tempMinMax, precip, data=clima).transform_filter(
  'datum.location == "Seattle"'
).resolve_scale(y='independent')

alt.LayerChart(...)

Por mais que exios duplos sejam uteis eles são _propensos a interpretações precipitadas_, já que diferentes unidades e escalas no mesmo eixo podem ser confuso. Já que é viável, é bom considerar transformações que mapeiam diferentes campos de dados em uma mesma unidade, poe exemplo mostrando  [quantiles](https://en.wikipedia.org/wiki/Quantile) ou mudança percentual relativa. 

## Faceta

*O fecetin* envolve a subdivisão de um conjunto de dados em grupos e a criação de um gráfico separado para cada grupo. Em notebooks anteriores, aprendemos como criar gráficos facetados usando os métodos `linha` e `coluna`. Vamos primeiro rever esses canais e depois mostrar como eles são instâncias do operador `facet` mais geral.

Vamos começar com um histograma básico de valores de temperatura máxima em Seattle:

In [15]:
alt.Chart(clima).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q')
)

alt.Chart(...)

_Como é que este perfil de temperatura muda com base no tempo de um determinado dia - isto é, se houve chuvisco, nevoeiro, chuva, neve ou sol?_

Vamos usar o método `column` para facetar os dados por tipo de clima.Também podemos usar `color` como uma codificação redundante, usando um intervalo de cores personalizado:

In [16]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart(clima).mark_bar().transform_filter(
  'datum.location == "Seattle"'
).encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors),
  alt.Column('weather:N')
).properties(
  width=150,
  height=150
)

alt.Chart(...)

Sem surpresa, os raros dias de neve concentram-se nas temperaturas mais frias, seguidos de dias de chuva e nevoeiro. Os dias de sol são mais quentes e, apesar dos estereótipos de Seattle, são os mais abundantes.No entanto, como qualquer habitante de Seattle lhe pode dizer, o chuvisco aparece ocasionalmente, independentemente da temperatura!_

Além dos métodos `row` e `column` *dentro* de uma definição de gráfico, podemos pegar uma definição básica de gráfico e aplicar facetes usando um operador `facet` explícito.

Vamos recriar o gráfico acima, mas desta vez usando `facet`.Começamos com a mesma definição básica de histograma, mas removemos a fonte de dados, a transformação do filtro e o canal da coluna. Podemos então invocar o método `facet`, passando os dados e especificando que devemos facetar as colunas de acordo com o campo `weather`. O método `facet` aceita ambos os argumentos `row` e `column`. Os dois podem ser usados juntos para criar uma grade 2D de gráficos facetados.

Finalmente, incluímos nossa transformação de filtro, aplicando-a ao gráfico facetado de nível superior. Embora pudéssemos aplicar a transformação do filtro à definição do histograma como antes, isso é um pouco menos eficiente. Em vez de filtrar os valores de “Nova York” dentro de cada célula de faceta, a aplicação do filtro ao gráfico facetado permite que o Vega-Lite saiba que podemos filtrar esses valores antecipadamente, antes da subdivisão da faceta.

In [17]:
colors = alt.Scale(
  domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
  range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
)

alt.Chart().mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=colors)
).properties(
  width=150,
  height=150
).facet(
  data=clima,
  column='weather:N'
).transform_filter(
  'datum.location == "Seattle"'
)

alt.FacetChart(...)

Dado todo o código extra acima, porque é que queremos usar um operador `facet` explícito? Para gráficos básicos, nós certamente devemos usar os métodos `column` ou `row` se pudermos. No entanto, usar o operador `facet` explicitamente é útil se quisermos facetar visualizações compostas, como gráficos em camadas.

Vamos revisitar nossos gráficos de temperatura em camadas de antes. Em vez de plotar os dados de Nova York e Seattle no mesmo gráfico, vamos dividi-los em facetas separadas. As definições dos gráficos individuais são praticamente as mesmas de antes: um gráfico de área e um gráfico de linhas. A única diferença é que, desta vez, não passaremos os dados diretamente para os construtores de gráficos; esperaremos e os passaremos para o operador de faceta mais tarde. Nós podemos colocar os gráficos em camadas como antes, e então invocar `facet` no objeto gráfico em camadas, passando os dados e especificando as facetas `column` baseadas no campo `location`:

In [18]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=clima,
  column='location:N'
)

alt.FacetChart(...)

Os gráficos facetados que vimos até agora utilizam os mesmos domínios de escala de eixo nas células facetadas.Esta predefinição de utilização de escalas e eixos *compartilhados* ajuda a uma comparação precisa dos valores.No entanto, em alguns casos, pode desejar escalar cada gráfico independentemente, por exemplo, se o intervalo de valores nas células for significativamente diferente.
À semelhança dos gráficos em camadas, os gráficos facetados também suportam a _resolução_ para escalas ou eixos independentes entre gráficos.Vamos ver o que acontece se chamarmos o método `resolver_eixo` para solicitar eixos y `independentes`:

In [19]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura méida (°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=clima,
  column='location:N'
).resolve_axis(y='independent')

alt.FacetChart(...)


_O gráfico acima parece praticamente inalterado, mas o gráfico de Seattle agora inclui seu próprio eixo.

E se ao invés disso nós chamarmos `resolve_scale` para resolver os domínios de escala subjacentes?

In [20]:
tempMinMax = alt.Chart().mark_area(opacity=0.3).encode(
  alt.X('month(date):T', title=None, axis=alt.Axis(format='%b')),
  alt.Y('average(temp_max):Q', title='Temperatura média(°C)'),
  alt.Y2('average(temp_min):Q'),
  alt.Color('location:N')
)

tempMid = alt.Chart().mark_line().transform_calculate(
  temp_mid='(+datum.temp_min + +datum.temp_max) / 2'
).encode(
  alt.X('month(date):T'),
  alt.Y('average(temp_mid):Q'),
  alt.Color('location:N')
)

alt.layer(tempMinMax, tempMid).facet(
  data=clima,
  column='location:N'
).resolve_scale(y='independent')

alt.FacetChart(...)

Agora vemos células de faceta com domínios de escala de eixos diferentes. Neste caso, usar escalas independentes parece ser uma má idéia! Os domínios não são muito diferentes e poderíamos ser levados a pensar que Nova Iorque e Seattle têm temperaturas máximas de verão semelhantes.

Para usar um cliché: só porque *pode* fazer algo, não significa que *deva*...

## 5.4 Concatenar

O *faceting* cria gráficos de [múltiplos pequenos](https://en.wikipedia.org/wiki/Small_multiple) que mostram subdivisões separadas dos dados. No entanto, podemos querer criar uma exibição de múltiplas visões com diferentes visões do *mesmo* conjunto de dados (não subconjuntos) ou visões envolvendo *diferentes* conjuntos de dados.

O Altair fornece operadores de *concatenação* para combinar gráficos arbitrários em um gráfico composto. O operador `hconcat` (abreviação `|`) realiza a concatenação horizontal, enquanto o operador `vconcat` (abreviação `&`) realiza a concatenação vertical.



Vamos começar com um gráfico de linha básico mostrando a temperatura máxima média por mês para Nova York e Seattle, assim como vimos antes:


In [21]:
alt.Chart(clima).mark_line().encode(
  alt.X('month(date):T', title=None),
  alt.Y('average(temp_max):Q'),
  color='local:N'
)

alt.Chart(...)

_E o que acontece se quisermos comparar não apenas a temperatura ao longo do tempo, mas também a precipitação e os níveis de vento?_

Vamos criar um gráfico concatenado consistindo de três gráficos. Começaremos definindo uma definição de gráfico "base" que contém todos os aspectos que devem ser compartilhados pelos nossos três gráficos. Podemos então modificar este gráfico base para criar variantes personalizadas, com diferentes codificações do eixo y para os campos `temp_max`, `precipitation` e `wind`. Podemos então concatená-los usando o operador abreviado de pipe (`|`):


In [22]:
base = alt.Chart(clima).mark_line().encode(
  alt.X('month(date):T', title=None),
  color='location:N'
).properties(
  width=240,
  height=180
)

temp = base.encode(alt.Y('average(temp_max):Q'))
precip = base.encode(alt.Y('average(precipitation):Q'))
wind = base.encode(alt.Y('average(wind):Q'))

temp | precip | wind

alt.HConcatChart(...)

Alternativamente, poderíamos usar o método mais explícito `alt.hconcat()` em vez do operador pipe `|`. _Tente reescrever o código acima para usar `hconcat` em vez disso._

A concatenação vertical funciona de maneira semelhante à concatenação horizontal. _Usando o operador `&` (ou o método `alt.vconcat`), modifique o código para usar uma ordenação vertical em vez de uma ordenação horizontal._

Por fim, note que concatenação horizontal e vertical podem ser combinadas. _O que acontece se você escrever algo como `(temp | precip) & wind`?_

_À parte_: Note a importância desses parênteses... o que acontece se você removê-los? Tenha em mente que esses operadores sobrecarregados ainda estão sujeitos às [regras de precedência de operadores do Python](https://docs.python.org/3/reference/expressions.html#operator-precedence), e, portanto, a concatenação vertical com `&` terá precedência sobre a concatenação horizontal com `|`!

Como vamos revisar mais tarde, os operadores de concatenação permitem combinar qualquer e todos os gráficos em um painel de múltiplas visões!


## 5.5 Repetir


Os operadores de concatenação acima são bastante gerais, permitindo que gráficos arbitrários sejam compostos. No entanto, o exemplo acima ainda era um pouco verboso: temos três gráficos muito semelhantes, mas temos que defini-los separadamente e depois concatená-los.

Para casos onde apenas uma ou duas variáveis estão mudando, o operador `repeat` oferece um atalho conveniente para criar múltiplos gráficos. Dada uma especificação de *template* com algumas variáveis livres, o operador repeat então criará um gráfico para cada atribuição especificada para essas variáveis.

Vamos recriar nosso exemplo de concatenação acima usando o operador `repeat`. O único aspecto que muda entre os gráficos é a escolha do campo de dados para o canal de codificação `y`. Para criar uma especificação de template, podemos usar a *variável repetidora* `alt.repeat('column')` como nosso campo do eixo y. Esse código simplesmente declara que queremos usar a variável atribuída ao repetidor `column`, que organiza gráficos repetidos em uma direção horizontal. (Como o repetidor fornece apenas o nome do campo, precisamos especificar separadamente o tipo de dados do campo como `type='quantitative'`.)

Em seguida, invocamos o método `repeat`, passando os nomes dos campos de dados para cada coluna:


In [23]:
alt.Chart(clima).mark_line().encode(
  alt.X('month(date):T',title=None),
  alt.Y(alt.repeat('column'), aggregate='average', type='quantitative'),
  color='location:N'
).properties(
  width=240,
  height=180
).repeat(
  column=['temp_max', 'precipitation', 'wind']
)

alt.RepeatChart(...)

A repetição é suportada tanto para colunas quanto para linhas. _O que acontece se você modificar o código acima para usar `row` em vez de `column`?_

Também podemos usar a repetição de `row` e `column` juntas! Uma visualização comum para análise exploratória de dados é a [matriz de gráficos de dispersão (ou SPLOM)](https://en.wikipedia.org/wiki/Scatter_plot#Scatterplot_matrices). Dada uma coleção de variáveis a serem analisadas, um SPLOM fornece uma grade de todos os gráficos pareados dessas variáveis, permitindo-nos avaliar associações potenciais.

Vamos usar o operador `repeat` para criar um SPLOM para os campos `temp_max`, `precipitation` e `wind`. Primeiro, criamos nossa especificação de template, com variáveis de repetição para os campos de dados dos eixos x e y. Em seguida, invocamos `repeat`, passando arrays de nomes de campos para usar tanto para `row` quanto para `column`. O Altair então gerará o [produto cruzado (ou, produto cartesiano)](https://en.wikipedia.org/wiki/Cartesian_product) para criar o espaço completo de gráficos repetidos:


In [24]:
alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=150,
  height=150
).repeat(
  data=clima,
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
).transform_filter(
  'datum.location == "Seattle"'
)

alt.RepeatChart(...)

_Olhando para esses gráficos, não parece haver uma associação forte entre precipitação e vento, embora vejamos que eventos extremos de vento e precipitação ocorrem em faixas de temperatura semelhantes (~5-15° C). No entanto, essa observação não é particularmente surpreendente: se revisarmos nosso histograma no início da seção de facetamento, podemos ver claramente que os dias com temperaturas máximas na faixa de 5-15° C são os que mais ocorrem._

*Modifique o código acima para obter uma melhor compreensão da repetição de gráficos. Tente adicionar outra variável (`temp_min`) ao SPLOM. O que acontece se você reorganizar a ordem dos nomes dos campos nos parâmetros `row` ou `column` para o operador `repeat`?*

_Por fim, para realmente apreciar o que o operador `repeat` oferece, reserve um momento para imaginar como você recriaria o SPLOM acima usando apenas `hconcat` e `vconcat`!_


## 5.6. A Álgebra da Composição Visual



Juntos, os operadores `layer`, `facet`, `concat` e `repeat` formam uma *álgebra de composição visual*: todos os operadores podem ser combinados de maneiras diferentes para formar uma variedade imensa de visualizações.

Como um exemplo, vamos começar fazendo dois gráficos básicos: um histograma e uma linha horizontal (marca do tipo `rule`) mostrando uma média global.

In [25]:
basic1 = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_bar().encode(
  alt.X('month(date):O'),
  alt.Y('average(temp_max):Q')
)

basic2 = alt.Chart(clima).transform_filter(
  'datum.location == "Seattle"'
).mark_rule(stroke='firebrick').encode(
  alt.Y('average(temp_max):Q')
)

basic1 | basic2

alt.HConcatChart(...)

Podemos combinar os dois gráficos usando um operador `layer`, e então usar `repeat` para fazer este mesmo histograma tratando agora a média de outros campos:

In [26]:
alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Mês'),
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('column'), aggregate='average', type='quantitative')
  )
).properties(
  width=200,
  height=150
).repeat(
  data=clima,
  column=['temp_max', 'precipitation', 'wind']
).transform_filter(
  'datum.location == "Seattle"'
)

alt.RepeatChart(...)

Focando apenas nos operadores de composição utilizados até agora, o modelo para nossa visualização seria:

repita([colunas])
|- ajunte em camadas
|  |- gráfico básico 1 (histograma)
|  |- gráfico básico 2 (linha horizontal)

```
repita([colunas])
|- ajunte em camadas
|  |- gráfico básico 1 (histograma)
|  |- gráfico básico 2 (linha horizontal)
```



Vamos agora explorar como podemos aplicar **todos** os operadores num [dashboard](https://pt.wikipedia.org/wiki/Painel_de_bordo) que nos dê uma visão geral do clima em Seattle. Vamos combinar a MSPLOT e histogramas facetados que criamos previamente com os histogramas repetidos que acabamos de fazer:

In [27]:
msplot = alt.Chart().mark_point(filled=True, size=15, opacity=0.5).encode(
  alt.X(alt.repeat('column'), type='quantitative'),
  alt.Y(alt.repeat('row'), type='quantitative')
).properties(
  width=125,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind'],
  column=['wind', 'precipitation', 'temp_max']
)

dateHist = alt.layer(
  alt.Chart().mark_bar().encode(
    alt.X('month(date):O', title='Mês'),
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  ),
  alt.Chart().mark_rule(stroke='firebrick').encode(
    alt.Y(alt.repeat('row'), aggregate='average', type='quantitative')
  )
).properties(
  width=175,
  height=125
).repeat(
  row=['temp_max', 'precipitation', 'wind']
)

tempHist = alt.Chart(clima).mark_bar().encode(
  alt.X('temp_max:Q', bin=True, title='Temperatura (°C)'),
  alt.Y('count():Q'),
  alt.Color('weather:N', scale=alt.Scale(
    domain=['drizzle', 'fog', 'rain', 'snow', 'sun'],
    range=['#aec7e8', '#c7c7c7', '#1f77b4', '#9467bd', '#e7ba52']
  ))
).properties(
  width=115,
  height=100
).facet(
  column='weather:N'
)

alt.vconcat(
  alt.hconcat(msplot, dateHist),
  tempHist,
  data=clima,
  title='Dashboard - Clima em Seattle'
).transform_filter(
  'datum.location == "Seattle"'
).resolve_legend(
  color='independent'
).configure_axis(
  labelAngle=0
)

alt.VConcatChart(...)

O modelo de composição completo para esse dashboard é:



```
concatene verticalmente
|- concatene horizontalmente
|  |- repita(linhas = [...], colunas = [...])
|  |  |- gráfico base do msplot
|  |- repita(linhas = [...])
|  |  |- ajunte em camadas
|  |  |  |- histograma base por datas 1
|  |  |  |- histograma base por datas 2
|- facete(coluna = 'weather')
|  |- histograma base por temperaturas
```



*Ufa!* Nossa dashboard também tem algumas customizações para melhorar o layout:




*   Ajustamos as propriedades de largura (`width`) e a altura (`height`) para melhorar o alinhamento dos gráficos e garantir que a visualização inteira cabe na tela;
*   Adicionamos `resolve_legend(color='independent')` para garantir que nossa legenda de cores seja associada apenas aos histogramas de temperatura, e não ao dashboard inteiro;
*   Ajustamos nossos eixos com `configure_axis(labelAngle=0)` para nos certificar que os títulos dos nossos eixos não sejam girados na concatenação, o que melhora o alinhamento dos gráficos do MSPLOT com os histogramas de datas, à direita.



*Experimente remover/modificar esses ajustes para ver como eles realmente afetam nosso dashboard!*

Esse mesmo dashboard também pode ser usado para representar dados de outros lugares na nossa base de dados. Tente modificar o código para que possamos ver dados de Nova York (`'New York'`) ao invés de Seattle.

## 5.7. Sumário

Para mais detalhes sobre composição de múltiplas visualizações, incluindo controle sobre espaçamento de "subgráficos" e títulos de cabeçalhos, veja a [documentação do Altair sobre gráficos compostos](https://altair-viz.github.io/user_guide/compound_charts.html).

Agora que sabemos compor multi-visualizações, podemos colocá-las em ação. Além de apresentar dados de forma estática, multi-visualização também nos permite criar interações de exploração multi-dimensional. Por exemplo, usando *seleções interligadas*, podemos destacar pontos em uma visualização para ver valores correspondentes em outras visualizações.

Na próxima seção, vamos examinar como criar *seleções de interatividade* para ambos gráficos individuais e visualizações múltiplas.